# Ship Model

> Library for ship model

In [ ]:

# | default_exp ship_model
%load_ext autoreload
%autoreload 2

In [ ]:
# | export
from dataclasses import dataclass
from typing import Union, TypeVar

import numpy as np

from scipy import optimize, interpolate

from ship_model_lib.ship_dimensions import (
    ShipDimensionsHollenbachSingleScrew,
    ShipDimensionsHollenbachTwinScrew,
    ShipDimensionsAddedResistance,
)
from ship_model_lib.calm_water_resistance import (
    CalmWaterResistanceBySpeedResistanceCurve,
    CalmWaterResistanceBySpeedPowerCurve,
    CalmWaterResistanceHollenbachSingleScrewDesignDraft,
    CalmWaterResistanceHollenbachTwinScrewDesignDraft,
    CalmWaterResistanceHollenbachSingleScrewBallastDraft,
)
from ship_model_lib.propulsor import (
    PropulsorOperatingPoint,
    PropulsorDataBseries,
    PropulsorDataOpenWater,
    PropulsorDataScalar,
    WakeFractionThrustDeductionFactorPoint,
    rps_to_rad_per_s,
)
from ship_model_lib.added_resistance import (
    WaveSpectrumType,
    AddedResistanceByStaWave2,
    AddedResistanceBySeaMarginCurve,
    AddedResistanceBySNNM,
    AddedResistanceWindITTC,
)
from ship_model_lib.operation_profile_structure import Weather, OperationPoint, Location
from ship_model_lib.utility import kn_to_m_per_s, m_per_s_to_kn, Interpolated1DValue
from ship_model_lib.machinery import (
    Point,
    Curve,
    EmissionType,
    EmissionFactor,
    FuelByMassFraction,
    FuelConsumption,
    PowerSourceWithEfficiency,
    PowerSourceWithSpecificFuelConsumption,
    MachinerySystem,
    MachinerySubsystemSimple,
    MachineryResult,
    MachinerySystemResult,
    LoadInput,
    PropulsionType,
)
from ship_model_lib.types import ShipType

In [ ]:
# | export

Numeric = TypeVar("Numeric", float, np.ndarray)


@dataclass
class ShipDescription:
    name: str
    type: ShipType


class HullOperatingPoint:
    def __init__(
        self,
        vessel_speed_kn: float,
        calm_water_resistance_newton: float,
        added_resistance_wave_newton: float,
        added_resistance_wind_newton: float,
    ):
        self.vessel_speed_kn = vessel_speed_kn
        self.calm_water_resistance_newton = calm_water_resistance_newton
        self.added_resistance_wave_newton = added_resistance_wave_newton
        self.added_resistance_wind_newton = added_resistance_wind_newton

    def __repr__(self):
        kws = [f"{key}={value!r}" for key, value in self.__dict__.items()]
        return "{}({})".format(type(self).__name__, ", ".join(kws))

    @property
    def total_resistance_newton(self):
        return (
            self.calm_water_resistance_newton
            + self.added_resistance_wave_newton
            + self.added_resistance_wind_newton
        )

    @property
    def total_towing_power_kw(self):
        vessel_speed_m_per_s = kn_to_m_per_s(self.vessel_speed_kn)
        return self.total_resistance_newton * vessel_speed_m_per_s / 1000


@dataclass
class HullData:
    b_beam_m: float
    lpp_length_between_perpendiculars_m: float
    los_length_over_surface_m: float
    lwl_length_water_line_m: float
    cb_block_coefficient: float
    ta_draft_aft_m: float
    tf_draft_forward_m: float
    wetted_surface_m2: float
    av_transverse_area_above_water_line_m2: float
    area_bilge_keel_m2: float


@dataclass
class PropulsorData:
    dp_diameter_propeller_m: float
    pd_pitch_diameter_ratio: float
    ear_blade_area_ratio: float
    z_blade_number: int


@dataclass
class ShipPerformanceData:
    ship_description: ShipDescription
    propeller_data: PropulsorOperatingPoint
    hull_data: HullOperatingPoint
    power_source_data: MachinerySystemResult


Accepted types for inputs to the ship class

In [ ]:
# | export

CalmWaterModel = Union[
    CalmWaterResistanceHollenbachTwinScrewDesignDraft,
    CalmWaterResistanceHollenbachSingleScrewBallastDraft,
    CalmWaterResistanceBySpeedResistanceCurve,
    CalmWaterResistanceHollenbachSingleScrewDesignDraft,
    CalmWaterResistanceBySpeedPowerCurve,
]
PropulsorModel = Union[
    PropulsorDataBseries, PropulsorDataOpenWater, PropulsorDataScalar
]
AddedResistanceWaveModel = Union[
    AddedResistanceByStaWave2, AddedResistanceBySeaMarginCurve, AddedResistanceBySNNM
]
PowerSystem = Union[MachinerySystem, MachinerySubsystemSimple]


## Ship model class
Handling all variants of different model implementations and returns a standard result class

In [ ]:
# | export


class ShipModel:
    def __init__(
        self,
        ship_description: ShipDescription = None,
        calm_water_resistance: CalmWaterModel = None,
        added_resistance_wave: AddedResistanceWaveModel = None,
        added_resistance_wind: AddedResistanceWindITTC = None,
        propulsor: PropulsorModel = None,
        machinery_system: MachinerySystem = None,
    ):
        if not ship_description:
            self.ship_description = ShipDescription(
                name="ship_name_unknown", type=ShipType("ship_type_unknown")
            )
        else:
            self.ship_description = ship_description
        self.calm_water_resistance = calm_water_resistance
        self.added_resistance_wave = added_resistance_wave
        self.added_resistance_wind = added_resistance_wind
        self.propulsor = propulsor
        self.machinery_system = machinery_system

    def _get_calm_water_power_speed_curve_interpolator(self):
        power_list_kw = []
        speed_list_kn = [*range(1, 30)]
        for speed_kn in speed_list_kn:
            if isinstance(
                self.calm_water_resistance, CalmWaterResistanceBySpeedPowerCurve
            ):
                power_list_kw.append(
                    self.calm_water_resistance.get_power_from_speed(speed_kn).value
                )
            else:
                power_list_kw.append(
                    kn_to_m_per_s(speed_kn)
                    * self.calm_water_resistance.get_resistance_from_speed(speed_kn)
                )
        return interpolate.interp1d(
            np.array(power_list_kw),
            np.array(speed_list_kn),
            kind="linear",
            fill_value="extrapolate",
        )

    def _iterate_ship_power(
        self,
        vessel_velocity_kn: float,
        power_goal_kw: float,
        weather: Weather,
        heading_deg: float,
        auxiliary_power_kw: float,
    ) -> float:
        """This is the function to find the speed that gives the required power numerically."""
        ship_performance_data = self.get_ship_performance_data_from_speed(
            vessel_speed_kn=vessel_velocity_kn,
            weather=weather,
            heading_deg=heading_deg,
            auxiliary_power_kw=auxiliary_power_kw,
        )
        if self.machinery_system is not None:
            power = ship_performance_data.power_source_data.total.power_on_source_kw
        elif self.propulsor is not None:
            power = ship_performance_data.propeller_data.shaft_power_kw
        else:
            power = ship_performance_data.hull_data.total_towing_power_kw
        return power - power_goal_kw

    @staticmethod
    def _fill_weather_with_default(weather: Weather) -> Weather:
        """Fill weather with default values if not provided."""
        if weather is None:
            return None
        if weather.significant_wave_height_m is None:
            weather.significant_wave_height_m = 0.0
        if weather.wave_direction_deg is None:
            weather.wave_direction_deg = 0.0
        if weather.wind_speed_m_per_s is None:
            weather.wind_speed_m_per_s = 0.0
        if weather.wind_direction_deg is None:
            weather.wind_direction_deg = 0.0
        return weather

    def get_ship_performance_data_from_speed(
        self,
        vessel_speed_kn,
        weather: Weather = None,
        heading_deg: float = None,
        auxiliary_power_kw: float = 0,
    ) -> ShipPerformanceData:
        # Calm water implementation selection logic
        weather = self._fill_weather_with_default(weather)
        hull_operating_point = HullOperatingPoint(
            vessel_speed_kn=vessel_speed_kn,
            calm_water_resistance_newton=0.0,
            added_resistance_wave_newton=0.0,
            added_resistance_wind_newton=0.0,
        )
        propulsor_operating_point = PropulsorOperatingPoint(
            vessel_speed_kn=vessel_speed_kn,
            n_rpm=0.0,
            j=0.0,
            wake_velocity_kn=0.0,
            propeller_thrust_newton=0.0,
            thrust_deduction_factor=0.0,
            resistance_newton=0.0,
            torque_newton_meter=0.0,
            efficiency_open_water=1.0,
        )
        shaft_power_kw = (
            0.0 if np.isscalar(vessel_speed_kn) else np.zeros_like(vessel_speed_kn)
        )
        # For semi-empirical models, we need to calculate the calm water resistance,
        # added resistance and propeller performance
        if (
            not isinstance(
                self.calm_water_resistance, CalmWaterResistanceBySpeedPowerCurve
            )
            and self.calm_water_resistance is not None
        ):
            calm_water_resistance_kilo_newton = (
                self.calm_water_resistance.get_resistance_from_speed(
                    velocity_kn=vessel_speed_kn
                )
            )
            if isinstance(calm_water_resistance_kilo_newton, Interpolated1DValue):
                calm_water_resistance_kilo_newton = (
                    calm_water_resistance_kilo_newton.value
                )
            hull_operating_point.calm_water_resistance_newton = (
                calm_water_resistance_kilo_newton * 1000
            )
            added_resistance_by_sea_margin = False
            if self.added_resistance_wave is not None and weather is not None:
                added_resistance_by_sea_margin = isinstance(
                    self.added_resistance_wave, AddedResistanceBySeaMarginCurve
                )
                if not added_resistance_by_sea_margin:
                    added_resistance_wave_newton = (
                        self.added_resistance_wave.get_added_resistance_newton(
                            vessel_speed_kn=vessel_speed_kn,
                            weather=weather,
                            heading_deg=heading_deg,
                        )
                    )
                else:
                    sea_margin_percent = (
                        self.added_resistance_wave.get_sea_margin_percent(
                            significant_wave_height_m=weather.significant_wave_height_m,
                        )
                    )
                    added_resistance_wave_newton = (
                        calm_water_resistance_kilo_newton
                        * sea_margin_percent
                        / 100
                        * 1000
                    )
            else:
                added_resistance_wave_newton = 0
            if (
                self.added_resistance_wind is not None
                and weather is not None
                and not added_resistance_by_sea_margin
            ):
                added_resistance_wind_newton = (
                    self.added_resistance_wind.get_added_resistance_newton(
                        vessel_speed_kn=vessel_speed_kn,
                        weather=weather,
                        heading_deg=heading_deg,
                    )
                )
            else:
                added_resistance_wind_newton = 0
            hull_operating_point.added_resistance_wave_newton = (
                added_resistance_wave_newton
            )
            hull_operating_point.added_resistance_wind_newton = (
                added_resistance_wind_newton
            )
            if self.propulsor is not None:
                propulsor_operating_point = self.propulsor.get_propulsor_data_from_vessel_speed_thrust(
                    vessel_speed_kn=vessel_speed_kn,
                    thrust_resistance_newton=hull_operating_point.total_resistance_newton,
                )
            shaft_power_kw = propulsor_operating_point.shaft_power_kw
        # For power curve models, we need to calculate the shaft power directly
        elif isinstance(
            self.calm_water_resistance, CalmWaterResistanceBySpeedPowerCurve
        ):
            shaft_power_kw = self.calm_water_resistance.get_power_from_speed(
                speed_kn=vessel_speed_kn
            ).value
            if np.isscalar(vessel_speed_kn):
                n_rps = 1 if vessel_speed_kn > 0 else 0
                torque_newton_meter = (
                    shaft_power_kw / rps_to_rad_per_s(n_rps) * 1000
                    if shaft_power_kw > 0
                    else 0
                )
            else:
                n_rps = np.zeros_like(vessel_speed_kn)
                n_rps[vessel_speed_kn > 0] = 1
                torque_newton_meter = np.zeros_like(vessel_speed_kn)
                torque_newton_meter[shaft_power_kw > 0] = shaft_power_kw[
                    shaft_power_kw > 0
                ] / rps_to_rad_per_s(n_rps[shaft_power_kw > 0]) * 1000
            propulsor_operating_point.n_rpm = n_rps * 60
            propulsor_operating_point.torque_newton_meter = torque_newton_meter

        # Machinery implementation selection logic
        if self.machinery_system is None:
            fuel_consumption = FuelConsumption(
                total_fuel_consumption=0,
                fuel_by_mass_fraction=FuelByMassFraction(marine_gas_oil=1),
            )
            power_source_operating_point = MachineryResult(
                power_on_source_kw=0, fuel_consumption=fuel_consumption
            )
        else:
            if self.machinery_system.propulsion_type == PropulsionType.MECHANICAL:
                mechanical_load_input = LoadInput(propulsion_load_kw=shaft_power_kw)
                electric_load_input = LoadInput(auxiliary_load_kw=auxiliary_power_kw)
            elif self.machinery_system.propulsion_type == PropulsionType.ELECTRIC:
                mechanical_load_input = None
                electric_load_input = LoadInput(
                    propulsion_load_kw=shaft_power_kw,
                    auxiliary_load_kw=auxiliary_power_kw,
                )
            else:
                raise NotImplementedError(
                    "Getting the result from the machinery system is not implemented for "
                    "the propulsion type: {self.machinery_system.propulsion_type}"
                )
            power_source_operating_point = self.machinery_system.get_machinery_result(
                mechanical_load=mechanical_load_input, electric_load=electric_load_input
            )
        return ShipPerformanceData(
            ship_description=self.ship_description,
            propeller_data=propulsor_operating_point,
            hull_data=hull_operating_point,
            power_source_data=power_source_operating_point,
        )

    def get_ship_performance_data_from_power(
        self,
        power_out_source_kw: Numeric,
        weather: Weather = None,
        heading_deg: float = None,
        auxiliary_power_kw: Numeric = 0,
    ) -> ShipPerformanceData:
        """Calculates the ship performance data from the power requirement.

        In this calculation, the ship speed and the propeller speed are calculated from the power
        requirement. It is a useful function for when there is an upper limit on the power available
        for the ship. Note that the power_kw is the power at the power source, internal combustion
        engine, fuel cells, gensets, etc. If no machinery system is defined, the power_kw is the
        shaft power at the propeller.

        :param power_out_source_kw: The power requirement at the power source if machinery system is
            defined. It is a propulsion power otherwise.
        :param weather: The weather conditions
        :param auxiliary_power_kw: The auxiliary power consumption (other than propulsor) in kW
        :return: The ship performance data
        """
        if np.isscalar(power_out_source_kw):
            power_out_source_kw = np.array([power_out_source_kw])
        if np.isscalar(auxiliary_power_kw):
            auxiliary_power_kw = np.ones(power_out_source_kw.shape) * auxiliary_power_kw

        vessel_speed_kn = []
        speed_estimator = self._get_calm_water_power_speed_curve_interpolator()
        for index, power_kw_scalar in enumerate(power_out_source_kw):
            initial_speed_estimate = speed_estimator(power_kw_scalar)
            sol = optimize.root_scalar(
                f=self._iterate_ship_power,
                x0=initial_speed_estimate,
                x1=initial_speed_estimate * 1.05,
                args=(power_kw_scalar, weather, heading_deg, auxiliary_power_kw[index]),
            )
            if sol.converged:
                speed_found = sol.root if np.isscalar(sol.root) else sol.root[0]
                vessel_speed_kn.append(speed_found)
            else:
                raise ValueError(
                    f"Could not find a solution for the power requirement of {power_kw_scalar} "
                    f"with the initial speed estimate of {initial_speed_estimate}."
                )
        vessel_speed_kn = np.array(vessel_speed_kn)
        return self.get_ship_performance_data_from_speed(
            vessel_speed_kn=vessel_speed_kn,
            weather=weather,
            auxiliary_power_kw=auxiliary_power_kw,
        )

    def get_ship_performance_data_from_operating_point(
        self, operation_point: OperationPoint
    ) -> ShipPerformanceData:
        """Calculates the ship performance data from the operating point.

        :param operation_point: The operating point
        :return: The ship performance data
        """
        performance_data = self.get_ship_performance_data_from_speed(
            vessel_speed_kn=operation_point.speed_kn,
            weather=operation_point.weather,
            heading_deg=operation_point.heading_deg,
            auxiliary_power_kw=operation_point.auxiliary_power,
        )
        # Determine what power source data is available
        if self.machinery_system is not None:
            if self.machinery_system.propulsion_type is PropulsionType.MECHANICAL:
                power_on_source_kw = (
                    performance_data.power_source_data.mechanical_system.power_on_source_kw
                )
            elif self.machinery_system.propulsion_type is PropulsionType.ELECTRIC:
                power_on_source_kw = (
                    performance_data.power_source_data.electric_system.power_on_source_kw
                )
            else:
                raise NotImplementedError(
                    "The power source data is not implemented for the propulsion type: {self.machinery_system.propulsion_type}"
                )
        elif self.machinery_system is None and self.propulsor is not None:
            power_on_source_kw = performance_data.propeller_data.shaft_power_kw
        elif self.machinery_system is None and self.propulsor is None:
            power_on_source_kw = performance_data.hull_data.total_towing_power_kw
        else:
            power_on_source_kw = np.zeros([len(operation_point.speed_kn)])

        greater_than_power_limit = np.greater(
            power_on_source_kw, operation_point.power_limit_kw
        )
        if np.any(greater_than_power_limit):
            for index, greater_than_power_limit_each in enumerate(
                greater_than_power_limit
            ):
                if greater_than_power_limit_each:
                    if operation_point.weather.significant_wave_height_m:
                        significant_wave_height_m = (
                            operation_point.weather.significant_wave_height_m[index]
                        )
                    else:
                        significant_wave_height_m = None
                    if operation_point.weather.mean_wave_period_s:
                        mean_wave_period_s = operation_point.weather.mean_wave_period_s[
                            index
                        ]
                    else:
                        mean_wave_period_s = (None,)
                    if operation_point.weather.wave_direction_deg:
                        wave_direction_deg = operation_point.weather.wave_direction_deg[
                            index
                        ]
                    else:
                        wave_direction_deg = None
                    if operation_point.weather.wind_speed_m_per_s:
                        wind_speed_m_per_s = operation_point.weather.wind_speed_m_per_s[
                            index
                        ]
                    else:
                        wind_speed_m_per_s = None
                    if operation_point.weather.wind_direction_deg:
                        wind_direction_deg = operation_point.weather.wind_direction_deg[
                            index
                        ]
                    else:
                        wind_direction_deg = None
                    if operation_point.weather.ocean_current_speed_m_per_s:
                        ocean_current_speed_m_per_s = (
                            operation_point.weather.ocean_current_speed_m_per_s[index]
                        )
                    else:
                        ocean_current_speed_m_per_s = None
                    if operation_point.weather.ocean_current_direction_deg:
                        ocean_current_direction_deg = (
                            operation_point.weather.ocean_current_direction_deg[index]
                        )
                    else:
                        ocean_current_direction_deg = None
                    weather = Weather(
                        significant_wave_height_m=significant_wave_height_m,
                        mean_wave_period_s=mean_wave_period_s,
                        wave_direction_deg=wave_direction_deg,
                        wind_speed_m_per_s=wind_speed_m_per_s,
                        wind_direction_deg=wind_direction_deg,
                        ocean_current_speed_m_per_s=ocean_current_speed_m_per_s,
                        ocean_current_direction_deg=ocean_current_direction_deg,
                    )
                    if operation_point.power_limit_kw:
                        power_limit_kw = operation_point.power_limit_kw[index]
                    else:
                        power_limit_kw = 1e6
                    if operation_point.heading_deg:
                        heading_deg = operation_point.heading_deg[index]
                    else:
                        heading_deg = None
                    if operation_point.auxiliary_power:
                        auxiliary_power_kw = operation_point.auxiliary_power[index]
                    else:
                        auxiliary_power_kw = 0
                    performance_data_from_power = (
                        self.get_ship_performance_data_from_power(
                            power_out_source_kw=power_limit_kw,
                            weather=weather,
                            heading_deg=heading_deg,
                            auxiliary_power_kw=auxiliary_power_kw,
                        )
                    )

                    operation_point.speed_kn[index] = (
                        performance_data_from_power.hull_data.vessel_speed_kn
                    )

            performance_data = self.get_ship_performance_data_from_speed(
                vessel_speed_kn=operation_point.speed_kn,
                weather=operation_point.weather,
                heading_deg=operation_point.heading_deg,
                auxiliary_power_kw=operation_point.auxiliary_power,
            )

        return performance_data

## Propeller implementations

In [ ]:
# | hide
import numpy as np
from ship_model_lib.ship_model import (
    WakeFractionThrustDeductionFactorPoint,
    PropulsorDataBseries,
    PropulsorDataOpenWater,
    PropulsorDataScalar,
)

In [ ]:
# | hide
pitch_diameter_ratio = 0.781
propeller_diameter = 9.81

vessel_speed_kn = np.array([13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18])
wake_factor = np.array(
    [0.338, 0.337, 0.336, 0.335, 0.334, 0.332, 0.329, 0.328, 0.326, 0.325, 0.322]
)
thrust_deduction = np.array(
    [0.201, 0.205, 0.209, 0.214, 0.218, 0.22, 0.223, 0.224, 0.227, 0.229, 0.233]
)

wake_fraction_thrust_deduction_values = [
    WakeFractionThrustDeductionFactorPoint(
        wake_fraction_factor=wake_factor_each,
        thrust_deduction_factor=thrust_deduction_each,
        vessel_speed_kn=vessel_speed_each,
    )
    for wake_factor_each, thrust_deduction_each, vessel_speed_each in zip(
        wake_factor, thrust_deduction, vessel_speed_kn
    )
]

Bseries

In [ ]:
# | hide
propulsor_data_bseries = PropulsorDataBseries(
    dp_diameter_propeller_m=propeller_diameter,
    pd_pitch_diameter_ratio=pitch_diameter_ratio,
    ear_blade_area_ratio=0.431,
    z_blade_number=4,
    wake_thrust_reduction=wake_fraction_thrust_deduction_values,
)

Open water propeller

In [ ]:
# | hide
propeller_curve_data = propulsor_data_bseries._generate_kt_kq_curve(j_step=0.1)
propulsor_data_open_water = PropulsorDataOpenWater(
    propeller_curve_points=propeller_curve_data,
    dp_diameter_propeller_m=propeller_diameter,
    wake_thrust_reduction=wake_fraction_thrust_deduction_values,
)

Scalar propeller

In [ ]:
# | hide
propulsor_data_scalar = PropulsorDataScalar(efficiency=0.7)

## Calm water implementations

Ship dimensions

In [ ]:
# | hide
from ship_model_lib.ship_model import ShipType, ShipDescription, ShipModel
from ship_model_lib.calm_water_resistance import (
    CalmWaterResistanceHollenbachSingleScrewDesignDraft,
    CalmWaterResistanceHollenbachTwinScrewDesignDraft,
    CalmWaterResistanceBySpeedPowerCurve,
    CalmWaterResistanceHollenbachSingleScrewBallastDraft,
    CalmWaterResistanceBySpeedResistanceCurve,
)
from ship_model_lib.added_resistance import (
    AddedResistanceByStaWave2,
    AddedResistanceBySeaMarginCurve,
    WaveSpectrumType,
)
from ship_model_lib.operation_profile_structure import Weather
from ship_model_lib.ship_dimensions import (
    ShipDimensionsHollenbachSingleScrew,
    ShipDimensionsHollenbachTwinScrew,
    ShipDimensionsAddedResistance,
)
from scipy import interpolate

In [ ]:
# | hide
b_beam_m = 24
lpp_length_between_perpendiculars_m = 145
los_length_over_surface_m = 150
lwl_length_water_line_m = 146.7
cb_block_coefficient = 0.75
ta_draft_aft_m = 8.2
tf_draft_forward_m = 8.2
wetted_surface_m2 = 4400
av_transverse_area_above_water_line_m2 = 2
area_bilge_keel_m2 = 1
kyy_radius_gyration_in_lateral_direction_non_dim = 0.26


HollenbachDesignDraftSingleScrew


In [ ]:
# | hide
ship_description = ShipDescription(name="Test vessel", type=ShipType.bulk_handysize)

ship_dimensions_single_screw = ShipDimensionsHollenbachSingleScrew(
    b_beam_m=b_beam_m,
    lpp_length_between_perpendiculars_m=lpp_length_between_perpendiculars_m,
    los_length_over_surface_m=los_length_over_surface_m,
    lwl_length_water_line_m=lwl_length_water_line_m,
    cb_block_coefficient=cb_block_coefficient,
    ta_draft_aft_m=ta_draft_aft_m,
    tf_draft_forward_m=tf_draft_forward_m,
    dp_diameter_propeller_m=propeller_diameter,
    wetted_surface_m2=wetted_surface_m2,
    av_transverse_area_above_water_line_m2=av_transverse_area_above_water_line_m2,
    area_bilge_keel_m2=area_bilge_keel_m2,
)

calm_water_resistance_hddss_kilo_newton = (
    CalmWaterResistanceHollenbachSingleScrewDesignDraft(
        ship_dimensions=ship_dimensions_single_screw
    )
)

HollenbachSingleScrewBallastDraft

In [ ]:
# | hide
calm_water_resistance_hbdss_kilo_newton = (
    CalmWaterResistanceHollenbachSingleScrewBallastDraft(
        ship_dimensions=ship_dimensions_single_screw
    )
)

HollenbachTwinScrewBallastDraft

In [ ]:
# | hide
ship_dimensions_twin_screw = ShipDimensionsHollenbachTwinScrew(
    b_beam_m=24,
    lpp_length_between_perpendiculars_m=145,
    los_length_over_surface_m=150,
    lwl_length_water_line_m=146.7,
    cb_block_coefficient=0.75,
    ta_draft_aft_m=8.2,
    tf_draft_forward_m=8.2,
    wetted_surface_m2=4400,
    av_transverse_area_above_water_line_m2=2,
    area_bilge_keel_m2=1,
    dp_diameter_propeller_m=propeller_diameter,
)
calm_water_resistance_hddts_kilo_newton = (
    CalmWaterResistanceHollenbachTwinScrewDesignDraft(
        ship_dimensions=ship_dimensions_twin_screw
    )
)

Speed Resistance curve

In [ ]:
# | hide
speed = np.linspace(13, 18.0, 10)

In [ ]:
# | hide
resistance_array = calm_water_resistance_hddss_kilo_newton.get_resistance_from_speed(
    speed
)
calm_water_resistance_speed_resistance_kn = CalmWaterResistanceBySpeedResistanceCurve(
    speed_ref_kn=speed, resistance_ref_k_n=resistance_array
)

Speed Power curve Power from resistance and Bseries propeller

In [ ]:
# | hide

wake_factor_intp = interpolate.PchipInterpolator(vessel_speed_kn, wake_factor)
thrust_reduction_intp = interpolate.PchipInterpolator(vessel_speed_kn, thrust_deduction)

resistance = (
    calm_water_resistance_hddss_kilo_newton.get_resistance_from_speed(speed) * 1000
)
power = propulsor_data_bseries.get_propulsor_data_from_vessel_speed_thrust(
    vessel_speed_kn=speed, thrust_resistance_newton=resistance
).shaft_power_kw

calm_water_resistance_speed_power_kilo_newton = CalmWaterResistanceBySpeedPowerCurve(
    speed_ref_kn=speed, power_ref_kw=power
)

## Added resistance implementations
StaWave2

In [ ]:
# | hide
ship_dimensions_stawave2 = ShipDimensionsAddedResistance(
    b_beam_m=b_beam_m,
    lpp_length_between_perpendiculars_m=lpp_length_between_perpendiculars_m,
    cb_block_coefficient=cb_block_coefficient,
    ta_draft_aft_m=ta_draft_aft_m,
    tf_draft_forward_m=tf_draft_forward_m,
    kyy_radius_gyration_in_lateral_direction_non_dim=0.26,
)
added_resistance_stawave2_newton = AddedResistanceByStaWave2(
    ship_dimension=ship_dimensions_stawave2,
    wave_spectrum_type=WaveSpectrumType.JONSWAP_ITTC_1984,
)
weather = Weather(
    significant_wave_height_m=3,
    mean_wave_period_s=10,
    wave_direction_deg=180,
    wind_speed_m_per_s=10,
    wind_direction_deg=160,
    ocean_current_speed_m_per_s=3,
    ocean_current_direction_deg=10,
)

## Machinery model implementations

Standard values

In [ ]:
from ship_model_lib.machinery import (
    FuelByMassFraction,
    Curve,
    Point,
    EmissionType,
    EmissionFactor,
    PowerSourceWithEfficiency,
    PowerLoad,
    PropulsionType,
    LoadInput,
    MachinerySubsystemSimple,
    MachinerySystem,
    MachineryResult,
    MachinerySystemResult,
)

In [ ]:
# | hide
fuel = FuelByMassFraction(hfo=1)
vessel_speed = 16.0
rated_power_kw = 10000.0
rated_power_aux_kw = 2000
scalar_efficiency = 0.5
scalar_efficiency_electric = 0.5
mechanical_system_efficiency = 0.95
electric_system_efficiency = 0.95
efficiency_curve = Curve(
    points=[
        Point(x=0.25, y=0.3),
        Point(x=0.5, y=0.4),
        Point(x=0.75, y=0.5),
        Point(x=1, y=0.35),
    ]
)

Emission models

In [ ]:
# | hide
factor_no_x_scalar = EmissionFactor(
    emission_type=EmissionType("nox"), factor=1.0, rated_power_kw=rated_power_kw
)

factor_co_2_scalar = EmissionFactor(
    emission_type=EmissionType("co2"), factor=2.0, rated_power_kw=rated_power_kw
)

factor_so_x_scalar = EmissionFactor(
    emission_type=EmissionType("sox"), factor=3.0, rated_power_kw=rated_power_kw
)
factor_pm_scalar = EmissionFactor(
    emission_type=EmissionType("pm"), factor=4.0, rated_power_kw=rated_power_kw
)

emission_factors_scalar = [
    factor_co_2_scalar,
    factor_no_x_scalar,
    factor_so_x_scalar,
    factor_pm_scalar,
]

nox_curve = Curve(
    [Point(x=0.25, y=6), Point(x=0.5, y=10), Point(x=0.75, y=12), Point(x=1, y=20)]
)
factor_no_x_curve = EmissionFactor(
    emission_type=EmissionType("nox"), factor=nox_curve, rated_power_kw=rated_power_kw
)

co2_curve = fuel.get_co_2_curve(efficiency=efficiency_curve)
factor_co_2_curve = EmissionFactor(
    emission_type=EmissionType("co2"), factor=co2_curve, rated_power_kw=rated_power_kw
)

so_x_curve = Curve(
    [Point(x=0.25, y=10), Point(x=0.5, y=7), Point(x=0.75, y=5), Point(x=1, y=8)]
)
factor_so_x_curve = EmissionFactor(
    emission_type=EmissionType("sox"), factor=so_x_curve, rated_power_kw=rated_power_kw
)

pm_curve = Curve(
    [
        Point(x=0.25, y=0.2),
        Point(x=0.5, y=0.15),
        Point(x=0.75, y=0.1),
        Point(x=1, y=0.1),
    ]
)
factor_pm_curve = EmissionFactor(
    emission_type=EmissionType("pm"), factor=pm_curve, rated_power_kw=rated_power_kw
)

emission_factors_curve = [
    factor_co_2_curve,
    factor_no_x_curve,
    factor_so_x_curve,
    factor_pm_curve,
]

Machinery model combinations

In [ ]:
# | hide
power_source_mechanical_system_scalar_no_emissions = PowerSourceWithEfficiency(
    fuel=fuel, efficiency=scalar_efficiency, rated_power_kw=rated_power_kw
)
power_source_mechanical_system_curve_no_emissions = PowerSourceWithEfficiency(
    fuel=fuel, efficiency=efficiency_curve, rated_power_kw=rated_power_kw
)
aux_power_source_electric_system_scalar_no_emissions = PowerSourceWithEfficiency(
    fuel=fuel, efficiency=scalar_efficiency_electric, rated_power_kw=rated_power_aux_kw
)
aux_power_source_electric_system_curve_no_emissions = PowerSourceWithEfficiency(
    fuel=fuel, efficiency=efficiency_curve, rated_power_kw=rated_power_aux_kw
)
power_source_mechanical_system_scalar_emissions_scalar = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=scalar_efficiency,
    rated_power_kw=rated_power_kw,
    emission_factors=emission_factors_scalar,
)
power_source_mechanical_system_curve_emissions_scalar = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=efficiency_curve,
    rated_power_kw=rated_power_kw,
    emission_factors=emission_factors_scalar,
)
aux_power_source_electric_system_scalar_emissions_scalar = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=scalar_efficiency_electric,
    rated_power_kw=rated_power_aux_kw,
    emission_factors=emission_factors_scalar,
)
aux_power_source_electric_system_curve_emissions_scalar = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=efficiency_curve,
    rated_power_kw=rated_power_aux_kw,
    emission_factors=emission_factors_scalar,
)
power_source_mechanical_system_scalar_emissions_curve = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=scalar_efficiency,
    rated_power_kw=rated_power_kw,
    emission_factors=emission_factors_curve,
)
power_source_mechanical_system_curve_emissions_curve = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=efficiency_curve,
    rated_power_kw=rated_power_kw,
    emission_factors=emission_factors_curve,
)
aux_power_source_electric_system_scalar_emissions_curve = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=scalar_efficiency_electric,
    rated_power_kw=rated_power_aux_kw,
    emission_factors=emission_factors_curve,
)
aux_power_source_electric_system_curve_emissions_curve = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=efficiency_curve,
    rated_power_kw=rated_power_aux_kw,
    emission_factors=emission_factors_curve,
)
main_power_source_electric_system_scalar_no_emissions = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=scalar_efficiency_electric,
    rated_power_kw=rated_power_kw + rated_power_aux_kw,
)
main_power_source_electric_system_curve_no_emissions = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=efficiency_curve,
    rated_power_kw=rated_power_kw + rated_power_aux_kw,
)
main_power_source_electric_system_scalar_emissions_scalar = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=scalar_efficiency_electric,
    rated_power_kw=rated_power_kw + rated_power_aux_kw,
    emission_factors=emission_factors_scalar,
)
main_power_source_electric_system_curve_emissions_scalar = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=efficiency_curve,
    rated_power_kw=rated_power_kw + rated_power_aux_kw,
    emission_factors=emission_factors_scalar,
)
main_power_source_electric_system_scalar_emissions_curve = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=scalar_efficiency_electric,
    rated_power_kw=rated_power_kw + rated_power_aux_kw,
    emission_factors=emission_factors_curve,
)
main_power_source_electric_system_curve_emissions_curve = PowerSourceWithEfficiency(
    fuel=fuel,
    efficiency=efficiency_curve,
    rated_power_kw=rated_power_kw + rated_power_aux_kw,
    emission_factors=emission_factors_curve,
)

mechanical_load = PowerLoad(efficiency=mechanical_system_efficiency)
electric_load = PowerLoad(efficiency=electric_system_efficiency)

mechanical_system_scalar_no_emissions = MachinerySubsystemSimple(
    power_source=power_source_mechanical_system_scalar_no_emissions,
    propulsion_load=mechanical_load,
)
aux_electric_system_scalar_no_emissions = MachinerySubsystemSimple(
    power_source=aux_power_source_electric_system_scalar_no_emissions,
    auxiliary_load=electric_load,
)
mechanical_system_curve_no_emissions = MachinerySubsystemSimple(
    power_source=power_source_mechanical_system_curve_no_emissions,
    propulsion_load=mechanical_load,
)
aux_electric_system_curve_no_emissions = MachinerySubsystemSimple(
    power_source=aux_power_source_electric_system_curve_no_emissions,
    auxiliary_load=electric_load,
)
mechanical_system_scalar_emissions_scalar = MachinerySubsystemSimple(
    power_source=power_source_mechanical_system_scalar_emissions_scalar,
    propulsion_load=mechanical_load,
)
aux_electric_system_scalar_emissions_scalar = MachinerySubsystemSimple(
    power_source=aux_power_source_electric_system_scalar_emissions_scalar,
    auxiliary_load=electric_load,
)
mechanical_system_scalar_emissions_curve = MachinerySubsystemSimple(
    power_source=power_source_mechanical_system_scalar_emissions_curve,
    propulsion_load=mechanical_load,
)
aux_electric_system_scalar_emissions_curve = MachinerySubsystemSimple(
    power_source=aux_power_source_electric_system_scalar_emissions_curve,
    auxiliary_load=electric_load,
)
mechanical_system_curve_emissions_scalar = MachinerySubsystemSimple(
    power_source=power_source_mechanical_system_curve_emissions_scalar,
    propulsion_load=mechanical_load,
)
aux_electric_system_curve_emissions_scalar = MachinerySubsystemSimple(
    power_source=aux_power_source_electric_system_curve_emissions_scalar,
    auxiliary_load=electric_load,
)
mechanical_system_curve_emissions_curve = MachinerySubsystemSimple(
    power_source=power_source_mechanical_system_curve_emissions_curve,
    propulsion_load=mechanical_load,
)
aux_electric_system_curve_emissions_curve = MachinerySubsystemSimple(
    power_source=aux_power_source_electric_system_curve_emissions_curve,
    auxiliary_load=electric_load,
)

main_electric_system_scalar_no_emissions = MachinerySubsystemSimple(
    power_source=main_power_source_electric_system_scalar_no_emissions,
    propulsion_load=electric_load,
    auxiliary_load=electric_load,
)

main_electric_system_curve_no_emissions = MachinerySubsystemSimple(
    power_source=main_power_source_electric_system_curve_no_emissions,
    propulsion_load=electric_load,
    auxiliary_load=electric_load,
)

main_electric_system_scalar_emissions_scalar = MachinerySubsystemSimple(
    power_source=main_power_source_electric_system_scalar_emissions_scalar,
    propulsion_load=electric_load,
    auxiliary_load=electric_load,
)

main_electric_system_curve_emissions_scalar = MachinerySubsystemSimple(
    power_source=main_power_source_electric_system_curve_emissions_scalar,
    propulsion_load=electric_load,
    auxiliary_load=electric_load,
)

main_electric_system_scalar_emissions_curve = MachinerySubsystemSimple(
    power_source=main_power_source_electric_system_scalar_emissions_curve,
    propulsion_load=electric_load,
    auxiliary_load=electric_load,
)

main_electric_system_curve_emissions_curve = MachinerySubsystemSimple(
    power_source=main_power_source_electric_system_curve_emissions_curve,
    propulsion_load=electric_load,
    auxiliary_load=electric_load,
)

machinery_system_scalar_no_emissions = MachinerySystem(
    propulsion_type=PropulsionType.MECHANICAL,
    mechanical_system=mechanical_system_scalar_no_emissions,
    electric_system=aux_electric_system_scalar_no_emissions,
)
machinery_system_curve_no_emissions = MachinerySystem(
    propulsion_type=PropulsionType.MECHANICAL,
    mechanical_system=mechanical_system_curve_no_emissions,
    electric_system=aux_electric_system_scalar_no_emissions,
)
machinery_system_scalar_emissions_scalar = MachinerySystem(
    propulsion_type=PropulsionType.MECHANICAL,
    mechanical_system=mechanical_system_scalar_emissions_scalar,
    electric_system=aux_electric_system_scalar_emissions_scalar,
)
machinery_system_curve_emissions_scalar = MachinerySystem(
    propulsion_type=PropulsionType.MECHANICAL,
    mechanical_system=mechanical_system_curve_emissions_scalar,
    electric_system=aux_electric_system_curve_emissions_scalar,
)
machinery_system_scalar_emissions_curve = MachinerySystem(
    propulsion_type=PropulsionType.MECHANICAL,
    mechanical_system=mechanical_system_scalar_emissions_curve,
    electric_system=aux_electric_system_scalar_emissions_curve,
)
machinery_system_curve_emissions_curve = MachinerySystem(
    propulsion_type=PropulsionType.MECHANICAL,
    mechanical_system=mechanical_system_curve_emissions_curve,
    electric_system=aux_electric_system_curve_emissions_curve,
)

machinery_system_electric_scalar_no_emissions = MachinerySystem(
    propulsion_type=PropulsionType.ELECTRIC,
    electric_system=main_electric_system_scalar_no_emissions,
)

machinery_system_electric_curve_no_emissions = MachinerySystem(
    propulsion_type=PropulsionType.ELECTRIC,
    electric_system=main_electric_system_curve_no_emissions,
)

machinery_system_electric_scalar_emissions_scalar = MachinerySystem(
    propulsion_type=PropulsionType.ELECTRIC,
    electric_system=main_electric_system_scalar_emissions_scalar,
)

machinery_system_electric_curve_emissions_scalar = MachinerySystem(
    propulsion_type=PropulsionType.ELECTRIC,
    electric_system=main_electric_system_curve_emissions_scalar,
)

machinery_system_electric_scalar_emissions_curve = MachinerySystem(
    propulsion_type=PropulsionType.ELECTRIC,
    electric_system=main_electric_system_scalar_emissions_curve,
)

machinery_system_electric_curve_emissions_curve = MachinerySystem(
    propulsion_type=PropulsionType.ELECTRIC,
    electric_system=main_electric_system_curve_emissions_curve,
)

machinery_system = [
    machinery_system_scalar_no_emissions,
    machinery_system_curve_no_emissions,
    machinery_system_scalar_emissions_scalar,
    machinery_system_curve_emissions_scalar,
    machinery_system_scalar_emissions_curve,
    machinery_system_curve_emissions_curve,
    machinery_system_electric_scalar_no_emissions,
    machinery_system_electric_curve_no_emissions,
    machinery_system_electric_scalar_emissions_scalar,
    machinery_system_electric_curve_emissions_scalar,
    machinery_system_electric_scalar_emissions_curve,
    machinery_system_electric_curve_emissions_curve,
]

# Testing ShipModel get_power_from_speed() function

## Implementing and testing MACHINERY model combinations using HollenbachDesignDraftSingleScrew, StaWave2 and Bseries

machinery_simple

machinery_system

In [ ]:
from pprint import pprint
from ship_model_lib.ship_model import ShipModel


results_ship_model_machinery_system = []
for index, machinery in enumerate(machinery_system):
    ship_model_machinery_system = ShipModel(
        ship_description=ship_description,
        calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
        added_resistance_wave=added_resistance_stawave2_newton,
        propulsor=propulsor_data_bseries,
        machinery_system=machinery,
    )
    result = ship_model_machinery_system.get_ship_performance_data_from_speed(
        vessel_speed_kn=vessel_speed, weather=weather, auxiliary_power_kw=500
    )
    results_ship_model_machinery_system.append(result)
    # pprint(result)
    # print('\n')

## Implementing and testing CALM WATER RESISTANCE MODEL implementations using StaWave2, machinery_system_curve_emissions_curve and Bseries

HollenbachSingleScrewBallastDraft

In [ ]:
# | hide
ship_model_calm_water_hbdss = ShipModel(
    ship_description=ship_description,
    calm_water_resistance=calm_water_resistance_hbdss_kilo_newton,
    added_resistance_wave=added_resistance_stawave2_newton,
    propulsor=propulsor_data_bseries,
    machinery_system=machinery_system_curve_emissions_curve,
)
results_calm_water_hbdss = (
    ship_model_calm_water_hbdss.get_ship_performance_data_from_speed(
        vessel_speed_kn=vessel_speed, weather=weather, auxiliary_power_kw=500
    )
)
# pprint(results_calm_water_hbdss)

Speed by resistance curve

In [ ]:
# | hide
ship_model_calm_water_speed_resistance = ShipModel(
    ship_description=ship_description,
    calm_water_resistance=calm_water_resistance_speed_resistance_kn,
    added_resistance_wave=added_resistance_stawave2_newton,
    propulsor=propulsor_data_bseries,
    machinery_system=machinery_system_curve_emissions_curve,
)
result_calm_water_speed_resistance = (
    ship_model_calm_water_speed_resistance.get_ship_performance_data_from_speed(
        vessel_speed_kn=vessel_speed, weather=weather, auxiliary_power_kw=500
    )
)
# pprint(result_calm_water_speed_resistance)

Speed power curve

In [ ]:
# | hide
ship_model_calm_water_speed_power = ShipModel(
    ship_description=ship_description,
    calm_water_resistance=calm_water_resistance_speed_power_kilo_newton,
    machinery_system=machinery_system_curve_emissions_curve,
)
result_calm_water_speed_power = (
    ship_model_calm_water_speed_power.get_ship_performance_data_from_speed(
        vessel_speed_kn=vessel_speed, weather=weather, auxiliary_power_kw=500
    )
)
# pprint(result_calm_water_speed_power)

## Implementing and testing PROPELLER MODEL implementations using StaWave2, machinery_system_curve_emissions_curve and HollenbachSingleScrewDesignDraft


Open water propeller

In [ ]:
# | hide
ship_model_open_water_propeller = ShipModel(
    ship_description=ship_description,
    calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
    added_resistance_wave=added_resistance_stawave2_newton,
    propulsor=propulsor_data_open_water,
    machinery_system=machinery_system_curve_emissions_curve,
)
result_open_water_propeller = (
    ship_model_open_water_propeller.get_ship_performance_data_from_speed(
        vessel_speed_kn=vessel_speed, weather=weather, auxiliary_power_kw=500
    )
)
# pprint(result_open_water_propeller)

Scalar propeller

In [ ]:
# | hide
ship_model_scalar_propeller = ShipModel(
    ship_description=ship_description,
    calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
    added_resistance_wave=added_resistance_stawave2_newton,
    propulsor=propulsor_data_scalar,
    machinery_system=machinery_system_curve_emissions_curve,
)
result_scalar_propeller = (
    ship_model_scalar_propeller.get_ship_performance_data_from_speed(
        vessel_speed_kn=vessel_speed, weather=weather, auxiliary_power_kw=500
    )
)
# pprint(result_scalar_propeller)

# Testing ShipModel get_speed_from_power() function

## Implementing and testing MACHINERY model combinations using HollenbachDesignDraftSingleScrew, StaWave2 and Bseries

Test by comparing get_ship_data_from_speed() with get_ship_performance_data_from_power() with machinery simple

In [ ]:
# | hide
test_speeds_kn = np.linspace(13.0, 17.4, 10)
for index, machinery in enumerate(machinery_system):
    for speed_kn in test_speeds_kn:
        ship_model_machinery_simple = ShipModel(
            ship_description=ship_description,
            calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
            added_resistance_wave=added_resistance_stawave2_newton,
            propulsor=propulsor_data_bseries,
            machinery_system=machinery,
        )
        result_speed = ship_model_machinery_simple.get_ship_performance_data_from_speed(
            vessel_speed_kn=speed_kn, weather=weather, auxiliary_power_kw=0
        )
        result_power = ship_model_machinery_simple.get_ship_performance_data_from_power(
            power_out_source_kw=result_speed.power_source_data.total.power_on_source_kw,
            weather=weather,
            auxiliary_power_kw=0,
        )
        assert np.isclose(
            result_speed.propeller_data.n_rpm,
            result_power.propeller_data.n_rpm,
            rtol=1e-3,
        ), (
            f"Speed calculated propeller rpm {result_speed.propeller_data.n_rpm} is not equal to power calculated propeller rpm {result_power.propeller_data.n_rpm} for speed {speed_kn} with machinery {machinery.propulsion_type.name},"
            f"propeller data speed result speed {result_speed.propeller_data.vessel_speed_kn}, power result speed {result_power.propeller_data.vessel_speed_kn},"
            f"propeller data speed result speed {result_speed.propeller_data.shaft_power_kw}, power result speed {result_power.propeller_data.shaft_power_kw},"
            f"Speed result: mechanical power {result_speed.power_source_data.mechanical_system.power_on_source_kw} electric power {result_speed.power_source_data.electric_system.power_on_source_kw} total power {result_speed.power_source_data.total.power_on_source_kw},"
        )

# Test different input combinations

Empty input

In [ ]:
# | hide
ship_model_no_input = ShipModel()
# pprint(ship_model_no_input.get_ship_performance_data_from_speed(vessel_speed_kn=vessel_speed))

Only calm water

In [ ]:
# | hide
ship_model_only_calm_water_input = ShipModel(
    calm_water_resistance=calm_water_resistance_hddss_kilo_newton
)
# pprint(ship_model_only_calm_water_input.get_ship_performance_data_from_speed(vessel_speed_kn=vessel_speed))

Calm water and added resistance without and with weather

In [ ]:
# | hide
ship_model_calm_water_added_resistance_input = ShipModel(
    calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
    added_resistance_wave=added_resistance_stawave2_newton,
)
# pprint(ship_model_calm_water_added_resistance_input.get_ship_performance_data_from_speed(vessel_speed_kn=vessel_speed))
# pprint(ship_model_calm_water_added_resistance_input.get_ship_performance_data_from_speed(vessel_speed_kn=vessel_speed, weather=weather))

## Test designed to evaluate array inputs

In [ ]:
# | hide
ones_array = np.ones_like(test_speeds_kn)
weather_array = Weather(
    significant_wave_height_m=ones_array * weather.significant_wave_height_m,
    mean_wave_period_s=ones_array * weather.mean_wave_period_s,
    wave_direction_deg=ones_array * weather.wave_direction_deg,
    wind_speed_m_per_s=ones_array * weather.wind_speed_m_per_s,
    wind_direction_deg=ones_array * weather.wave_direction_deg,
    ocean_current_speed_m_per_s=ones_array * weather.ocean_current_speed_m_per_s,
    ocean_current_direction_deg=ones_array * weather.ocean_current_direction_deg,
)
electric_power_list = np.array([500] * len(test_speeds_kn))
for index, machinery in enumerate(machinery_system):
    ship_model_machinery_simple = ShipModel(
        ship_description=ship_description,
        calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
        propulsor=propulsor_data_bseries,
        machinery_system=machinery,
    )
    result_speed = ship_model_machinery_simple.get_ship_performance_data_from_speed(
        vessel_speed_kn=test_speeds_kn,
        weather=weather_array,
        auxiliary_power_kw=electric_power_list,
    )
    result_power = ship_model_machinery_simple.get_ship_performance_data_from_power(
        power_out_source_kw=result_speed.power_source_data.total.power_on_source_kw,
        weather=weather_array,
        auxiliary_power_kw=electric_power_list,
    )
    assert np.allclose(
        result_speed.propeller_data.vessel_speed_kn,
        result_power.propeller_data.vessel_speed_kn,
        rtol=1e-25,
    ), f"The results are not the same for propeller speed: {result_speed.propeller_data.vessel_speed_kn} vs {result_power.propeller_data.vessel_speed_kn}"

## Test get_ship_data_from_operating_point

### Test with mechanical machinery system and propeller

In [ ]:
# | hide
power_limit = 6000.0
power_limit_test_speeds = np.linspace(8.0, 17.4, 10)

for machinery in machinery_system:
    ship_model_machinery_simple_power_limit_test = ShipModel(
        ship_description=ship_description,
        calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
        added_resistance_wave=added_resistance_stawave2_newton,
        propulsor=propulsor_data_bseries,
        machinery_system=machinery,
    )

    speed_limited_power = ship_model_machinery_simple_power_limit_test.get_ship_performance_data_from_power(
        power_out_source_kw=power_limit, weather=weather, auxiliary_power_kw=0
    )

    for speed_kn in power_limit_test_speeds:
        operation_point = OperationPoint(
            speed_kn=speed_kn,
            power_limit_kw=power_limit,
            auxiliary_power=0,
            weather=weather,
        )
        result_power = ship_model_machinery_simple_power_limit_test.get_ship_performance_data_from_operating_point(
            operation_point=operation_point
        )
        # pprint(f"Speed setpoint: {speed_kn} - Speed achieved: {result_power.hull_data.vessel_speed_kn} - Power: {result_power.power_source_data.total.power_on_source_kw}")
        assert np.less_equal(
            result_power.power_source_data.total.power_on_source_kw, power_limit + 0.1
        ), f"Power achieved {result_power.power_source_data.total.power_on_source_kw} is not equal or less than the power limited {power_limit}"
        assert np.less_equal(
            result_power.hull_data.vessel_speed_kn,
            speed_limited_power.hull_data.vessel_speed_kn,
        ), f"Speed achieved {result_power.hull_data.vessel_speed_kn} is not equal to speed limited {speed_limited_power.hull_data.vessel_speed_kn}"

### Test without machinery system

In [ ]:
# | hide
power_limit = 6000.0
power_limit_test_speeds = np.linspace(8.0, 17.4, 10)

ship_model_machinery_simple_power_limit_test = ShipModel(
    ship_description=ship_description,
    calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
    added_resistance_wave=added_resistance_stawave2_newton,
    propulsor=propulsor_data_bseries,
    machinery_system=None,
)

speed_limited_power = (
    ship_model_machinery_simple_power_limit_test.get_ship_performance_data_from_power(
        power_out_source_kw=power_limit, weather=weather, auxiliary_power_kw=0
    )
)

for speed_kn in power_limit_test_speeds:
    operation_point = OperationPoint(
        speed_kn=speed_kn,
        power_limit_kw=power_limit,
        auxiliary_power=0,
        weather=weather,
    )
    result_power = ship_model_machinery_simple_power_limit_test.get_ship_performance_data_from_operating_point(
        operation_point=operation_point
    )
    # pprint(f"Speed setpoint: {speed_kn} - Speed achieved: {result_power.hull_data.vessel_speed_kn} - Power: {result_power.propeller_data.shaft_power_kw}")
    assert np.less_equal(
        result_power.propeller_data.shaft_power_kw, power_limit + 0.1
    ), f"Power achieved {result_power.propeller_data.shaft_power_kw} is not equal or less than the power limited {power_limit}"
    assert np.less_equal(
        result_power.hull_data.vessel_speed_kn,
        speed_limited_power.hull_data.vessel_speed_kn,
    ), f"Speed achieved {result_power.hull_data.vessel_speed_kn} is not equal to speed limited {speed_limited_power.hull_data.vessel_speed_kn}"

### Test without propeller

In [ ]:
# | hide
power_limit = 6000.0
power_limit_test_speeds = np.linspace(8.0, 17.4, 10)

ship_model_machinery_simple_power_limit_test = ShipModel(
    ship_description=ship_description,
    calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
    added_resistance_wave=added_resistance_stawave2_newton,
    propulsor=None,
    machinery_system=None,
)

speed_limited_power = (
    ship_model_machinery_simple_power_limit_test.get_ship_performance_data_from_power(
        power_out_source_kw=power_limit, weather=weather, auxiliary_power_kw=0
    )
)

for speed_kn in power_limit_test_speeds:
    operation_point = OperationPoint(
        speed_kn=speed_kn,
        power_limit_kw=power_limit,
        auxiliary_power=0,
        weather=weather,
    )
    result_power = ship_model_machinery_simple_power_limit_test.get_ship_performance_data_from_operating_point(
        operation_point=operation_point
    )
    # pprint(f"Speed setpoint: {speed_kn} - Speed achieved: {result_power.hull_data.vessel_speed_kn} - Power: {result_power.hull_data.total_towing_power_kw}")
    assert np.less_equal(
        result_power.hull_data.total_towing_power_kw, power_limit + 0.1
    ), f"Power achieved {result_power.hull_data.total_towing_power_kw} is not equal or less than the power limited {power_limit}"
    assert np.less_equal(
        result_power.hull_data.vessel_speed_kn,
        speed_limited_power.hull_data.vessel_speed_kn,
    ), f"Speed achieved {result_power.hull_data.vessel_speed_kn} is not equal to speed limited {speed_limited_power.hull_data.vessel_speed_kn}"

# Testing ShipModel get_ship_performance_data_from_operating_point() function

## Generating operation points. 

In [ ]:
# | hide
from datetime import datetime
import pandas as pd

df_voyage = pd.read_pickle("test_data/df_filtered_voyages.pkl")
voyage_list = df_voyage.iloc[0]["voyage_track_geometry"]

In [ ]:
# | hide
time_stamp = voyage_list[0][1]
time_object = datetime.strptime(time_stamp, "%Y-%m-%dT%H:%M:%S")

one_operation_point = OperationPoint(
    timestamp_seconds=time_object.timestamp(),
    location=Location(latitude=voyage_list[0][3], longitude=voyage_list[0][2]),
    heading_deg=voyage_list[0][4],
    speed_kn=voyage_list[0][5],
)
# pprint(one_operation_point.to_dict())

In [ ]:
# | hide
operation_list = []
for i in range(1, len(df_voyage) + 1):
    voyage_list = df_voyage.iloc[i - 1]["voyage_track_geometry"]
    time_stamp = np.array(
        [
            datetime.strptime(point[1], "%Y-%m-%dT%H:%M:%S").timestamp()
            for point in voyage_list
        ]
    )
    longitude = np.array([point[2] for point in voyage_list])
    latitude = np.array([point[3] for point in voyage_list])
    heading = np.array([point[4] for point in voyage_list])
    speed = np.array([point[5] for point in voyage_list])
    location = Location(latitude=latitude, longitude=longitude)

    operation = OperationPoint(
        timestamp_seconds=time_stamp,
        location=location,
        heading_deg=heading,
        speed_kn=speed,
    )

    operation_list.append(operation)

# pprint(operation_list[0].to_dict())

In [ ]:
# | hide
operation_points_array = operation_list[0]

## Testing with only HollenbachDesignDraftSingleScrew

In [ ]:
# | hide
ship_model_operation_point_hull_only = ShipModel(
    calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
)

result_operation_point_hull_only_single_point = (
    ship_model_operation_point_hull_only.get_ship_performance_data_from_operating_point(
        operation_point=one_operation_point
    )
)
# pprint(result_operation_point_hull_only_single_point)

In [ ]:
# | hide
ship_model_operation_point_hull_only = ShipModel(
    calm_water_resistance=calm_water_resistance_hddss_kilo_newton,
)

result_operation_point_hull_only_array = (
    ship_model_operation_point_hull_only.get_ship_performance_data_from_operating_point(
        operation_point=operation_points_array
    )
)
# pprint(result_operation_point_hull_only_array)